In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQjf_HNeEZKM-XJX-q5v4cfNrB3kcv4gOT8kFbV9rurfoX_H5Qv9112Pv0PgYNFSzbReyNlQkLrJib3/pubhtml?utm_source=google&utm_medium=cpc&utm_campaign=%E6%B8%85%E5%86%A0%E4%B8%80%E8%99%9F%E5%8B%95%E6%85%8B%E8%A1%A8#'
resp = requests.get(url).text

print(resp)

In [26]:

soup = BeautifulSoup(resp, 'html.parser')

# print(soup.prettify())
# print(soup.table)
# print(str(soup.table))
# soup = soup.table
# pd.read_html(soup.table, encoding='utf-8')[0]

a = pd.read_html(str(soup.table), encoding='utf-8')[0]
# a.columns = a.iloc[:2, :]
# a.iloc[:2, :]
a.columns=pd.MultiIndex.from_arrays(a.iloc[0:2].values)
a['科達']
a

1.0   NaN  最新用藥指引  全聯會提供公費清冠一號民眾意見反應專區   順天堂   莊松榮    立康    勝昌   勸奉堂  \
         2.0    編號     縣市別               醫療院所名稱  剩餘人次  剩餘人次  剩餘人次  剩餘人次  剩餘人次   
0        1.0   NaN  最新用藥指引  全聯會提供公費清冠一號民眾意見反應專區   順天堂   莊松榮    立康    勝昌   勸奉堂   
1        2.0    編號     縣市別               醫療院所名稱  剩餘人次  剩餘人次  剩餘人次  剩餘人次  剩餘人次   
2        NaN   NaN     NaN                  NaN   NaN   NaN   NaN   NaN   NaN   
3        3.0   835     南投縣              鼎億堂中醫診所     0     1     0     0    50   
4        4.0   826     南投縣               永安中醫診所     0    90     0     0     0   
...      ...   ...     ...                  ...   ...   ...   ...   ...   ...   
1553  1553.0  1509     高雄市               祐祥中醫診所     0     0     0     0     0   
1554  1554.0  1573     高雄市      義大醫療財團法人義大癌治療醫院     0     0     0     0     0   
1555  1555.0  1588     高雄市               五乙中醫診所     0     0     0     0     0   
1556  1556.0  1589     高雄市               祥霖中醫診所     0     0     0     0     0   
1557  1557.0  1602     高雄市               濟民中醫診所     0     0     0     0     0   

        華陀  ...    科達    富田  2022/8/16 下午 10:18:29  原廠清冠一號照片  清冠一號Q&A收錄集  \
      剩餘人次  ...  剩餘人次  剩餘人次             醫療院所最新更新時間    清冠一號公費      清冠一號自費   
0       華陀  ...    科達    富田  2022/8/16 下午 10:18:29  原廠清冠一號照片  清冠一號Q&A收錄集   
1     剩餘人次  ...  剩餘人次  剩餘人次             醫療院所最新更新時間    清冠一號公費      清冠一號自費   
2      NaN  ...   NaN   NaN                    NaN       NaN         NaN   
3        0  ...     0     0   2022/8/16 下午 2:30:20         是         NaN   
4        5  ...     0     0  2022/8/16 上午 11:36:17         是         NaN   
...    ...  ...   ...   ...                    ...       ...         ...   
1553     0  ...     0     0  2022/5/20 上午 12:00:00       NaN         NaN   
1554     0  ...     0     0  2022/5/20 上午 12:00:00       NaN         NaN   
1555     0  ...     0     0  2022/5/20 上午 12:00:00       NaN         NaN   
1556     0  ...     0     0  2022/5/20 上午 12:00:00       NaN         NaN   
1557     0  ...     0     0  2022/5/20 上午 12:00:00       NaN         NaN   

          NaN           就診中醫院所前請先點選西醫快篩陽性判斷視訊院所               NaN  \
      星期日特約門診  診療長新冠症狀           下方為中醫院所LINE ID                電話   
0         NaN      NaN  就診中醫院所前請先點選西醫快篩陽性判斷視訊院所               NaN   
1     星期日特約門診  診療長新冠症狀           下方為中醫院所LINE ID                電話   
2         NaN      NaN                      NaN               NaN   
3         NaN        是      LINE ID: 0921887659       049-2642557   
4         NaN        是               0492323166       049-2323166   
...       ...      ...                      ...               ...   
1553      NaN      NaN                      NaN        07-3657888   
1554      NaN      NaN                      NaN  07-6150022分機6112   
1555      NaN      NaN                      NaN      (07) 3526051   
1556      NaN      NaN                      NaN      (07) 3946788   
1557      NaN      NaN                      NaN        07-2410822   

                       
                   地址  
0                 NaN  
1                  地址  
2                 NaN  
3     南投縣竹山鎮集山路三段213號  
4      南投縣草屯鎮明賢街一段28號  
...               ...  
1553  高雄市楠梓區右昌街718號1樓  
1554     高雄市燕巢區義大路21號  
1555     高雄市楠梓區興楠路17號  
1556    高雄市三民區聯興路125號  
1557    高雄市新興區中山一路61號  

[1558 rows x 23 columns]